In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms as T
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import nrrd
import pandas as pd
import os
import sys

# sys.path.append("/mnt/raid/C1_ML_Analysis/source/autoencoder/src")
sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/")
sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/nets")

from nets import diffusion, spade, lotus, cut, layers, cut_G
from loaders import ultrasound_dataset as usd


In [ ]:
# class ConditionalInstanceNorm2d(nn.Module):
#     def __init__(self, num_features, num_classes):
#         super().__init__()
#         self.instance_norm = nn.InstanceNorm2d(num_features, affine=False)
#         self.gamma_embed = nn.Embedding(num_classes, num_features)
#         self.beta_embed = nn.Embedding(num_classes, num_features)
#         # Initialize scale close to 1 and bias as 0.
#         nn.init.constant_(self.gamma_embed.weight, 1)
#         nn.init.constant_(self.beta_embed.weight, 0)
    
#     def forward(self, x, labels):
#         out = self.instance_norm(x)
#         gamma = self.gamma_embed(labels).unsqueeze(2).unsqueeze(3)
#         beta = self.beta_embed(labels).unsqueeze(2).unsqueeze(3)
#         return gamma * out + beta

In [ ]:
# cond = ConditionalInstanceNorm2d(3, 10)

In [ ]:


batch_size = 8
dm = usd.Cut3DDataModule(
    csv_train_diffusor="/mnt/raid/C1_ML_Analysis/simulated_data_export/animation_export/all_poses_sub_train_train.csv", 
    csv_valid_diffusor="/mnt/raid/C1_ML_Analysis/simulated_data_export/animation_export/all_poses_sub_train_test.csv",
    
    mount_point="/mnt/raid/C1_ML_Analysis/simulated_data_export/animation_export",
    batch_size=batch_size,
    num_workers=1,
    img_column="file_path",
    class_column="pred_class",
    prefetch_factor=2,
    drop_last=False,
    )
dm.setup()

In [ ]:
train_dl = dm.train_dataloader()

In [ ]:
import torch
from torchvision.utils import make_grid
import plotly.express as px

def plot_tensor_grid(tensor, nrow=4, title="Image Grid", scale=1.0):
    """
    Uses torchvision's make_grid to arrange images into a grid, then displays it with Plotly.

    Args:
        tensor: PyTorch tensor of shape [B, N, 1, H, W]
        nrow: Number of images per row in the grid
        title: Plot title
    """
    B, N, C, H, W = tensor.shape
    assert C == 1, "Only grayscale images (C=1) are supported."

    # Flatten batch and image dimensions
    tensor = tensor.view(B * N, C, H, W)

    # Create image grid
    grid = make_grid(tensor, nrow=nrow, padding=2, normalize=True, scale_each=True)

    # Convert to numpy for Plotly
    grid_np = grid.permute(1,2,0).cpu().numpy()  # shape [H, W] since it's grayscale

    # Determine scaled dimensions
    height, width, C = grid_np.shape
    display_height = int(height * scale)
    display_width = int(width * scale)

    # Plot with Plotly
    fig = px.imshow(grid_np, color_continuous_scale='gray')
    fig.update_layout(
        title=title,
        coloraxis_showscale=False,
        width=display_width,
        height=display_height
    )
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()


In [ ]:
batch = next(iter(train_dl))
plot_tensor_grid(torch.stack(batch, dim=0), nrow=4, title='Image Grid')



In [ ]:
model = cut_G.ConditionalGenerator(in_channels=1, features=64, residuals=9, num_classes=4)

In [ ]:
x, patch_ids = model(input=torch.rand(1, 1, 256, 256), labels=torch.randint(low=0, high=4, size=(1,)), encode_only=True)

In [ ]:
y, _ = model(input=torch.rand(1, 1, 256, 256), labels=torch.randint(low=0, high=4, size=(1,)), encode_only=True, patch_ids=patch_ids)

In [ ]:
X = batch
X = torch.stack(batch, dim=0)
        
X_labels = torch.arange(0, len(X))
permuted_idx = torch.randperm(X.shape[0])

Y = X[permuted_idx]
Y_labels = X_labels[permuted_idx]

X_labels = X_labels.repeat(X.shape[1])
Y_labels = Y_labels.repeat(X.shape[1])

print(X.shape, X_labels.shape)
print(Y.shape, Y_labels.shape)

print(X_labels)
print(Y_labels)

X = X.view(X.shape[0]*X.shape[1], X.shape[2], X.shape[3], X.shape[4]).contiguous()
Y = Y.view(Y.shape[0]*Y.shape[1], Y.shape[2], Y.shape[3], Y.shape[4]).contiguous()



In [ ]:
model(X, labels=Y_labels)